In [1]:
print("Welcome")

Welcome


In [5]:
import spacy
import pickle
import random
import json

In [2]:
import os

os.getcwd()

'c:\\Users\\arshi\\Downloads\\Desktop\\Bro-Project\\SkillForge.ai\\research'

In [3]:
os.chdir('c:\\Users\\arshi\\Downloads\\Desktop\\Bro-Project\\SkillForge.ai')

In [4]:
os.getcwd()

'c:\\Users\\arshi\\Downloads\\Desktop\\Bro-Project\\SkillForge.ai'

In [90]:
file_path = 'data\\dataset.json'

In [91]:
with open(file_path, 'r') as file:
    train_data = json.load(file)

In [92]:
len(train_data)

1014

In [93]:
text_sample = train_data[0]

In [94]:
text_sample

['\xa0 \xa0\nContact\nwww.linkedin.com/in/omjagri\n(LinkedIn)\nTop Skills\nphp\nMySQL\nJavaScript\nCertifications\nPhp & Js Om Prakash Jagri\nFull Stack Developer | PHP | Laravel | Vue Js\nKathmandu, Bāgmatī, Nepal\nSummary\nExperienced Developer with a demonstrated history of working in\nthe information technology and services industry. Skilled in Laravel,\nPHP, Cascading Style Sheets (CSS), JavaScript, vue js and MySQL.\nStrong engineering professional with a B.sc.CSIT(Bachelors of\nScience in Computer Science and Information Technology) focused\nin Computer Science from Tribhuvan University, Institute of Science\n& Tchnology. \nExperience\nSearchable Design LLC\nSoftware Developer\nJune 2021\xa0-\xa0Present\xa0 (1 year 7 months)\nNepal\nFull Stack Developer Laravel with Vue Js\nBenekiva\nTechnical Documentation\nSeptember 2021\xa0-\xa0Present\xa0 (1 year 4 months)\nUnited States\nBidhee\n3 years 9 months\nLaravel Developer\nMarch 2018\xa0-\xa0May 2021\xa0 (3 years 3 months)\nBaneswa

In [95]:
def convert_json_to_spacy_format(json_data):
    """
    Convert JSON resume data into spaCy NER training format.
    
    Args:
        json_data (list): List of dictionaries containing resume data in the provided JSON format.
        
    Returns:
        list: List of tuples in (text, annotations) format suitable for spaCy NER training.
    """
    training_data = []
    
    for entry in json_data:
        if not isinstance(entry, list) or len(entry) != 2:
            continue
            
        text = entry[0]
        entities = entry[1].get("entities", [])
        
        # Convert entity format from [start, end, label] to (start, end, label)
        formatted_entities = []
        for entity in entities:
            if len(entity) == 3:
                formatted_entities.append(tuple(entity))
        
        training_data.append((text, {"entities": formatted_entities}))
    
    return training_data

In [96]:
convert_data = convert_json_to_spacy_format(train_data)

In [97]:
len(convert_data)

1014

In [98]:
from spacy.training import Example
from spacy.util import minibatch, compounding
from pathlib import Path

In [99]:
nlp = spacy.blank('en')

In [100]:
def train_ner_model(training_data, model_name="resume_ner", output_dir=None, n_iter=100):
    """
    Train a spaCy NER model with the provided training data.
    
    Args:
        training_data (list): List of training examples in (text, annotations) format
        model_name (str): Name for the new model
        output_dir (str): Directory to save the trained model (None to not save)
        n_iter (int): Number of training iterations
        
    Returns:
        spacy.Language: The trained NER model
    """
    # Load a blank English model or existing model
    try:
        nlp = spacy.load(model_name)
        print(f"Loaded existing model '{model_name}'")
    except:
        nlp = spacy.blank("en")
        print(f"Created blank 'en' model")
    
    # Add NER pipeline if it doesn't exist
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    else:
        ner = nlp.get_pipe("ner")
    
    # Add entity labels to the NER component
    for _, annotations in training_data:
        for ent in annotations.get("entities", []):
            ner.add_label(ent[2])
    
    # Disable other pipelines during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        # Initialize the model with random weights
        nlp.initialize(lambda: [Example.from_dict(nlp.make_doc(text), annotations) 
                      for text, annotations in training_data])
        
        # Training loop
        print("Beginning training...")
        for itn in range(n_iter):
            random.shuffle(training_data)
            losses = {}
            
            # Batch the examples using spaCy's minibatch
            batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                examples = []
                for text, annotations in batch:
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    examples.append(example)
                
                nlp.update(examples, drop=0.5, losses=losses)
            
            print(f"Iteration {itn + 1}/{n_iter}, Loss: {losses['ner']:.4f}")
    
    # Save the model if output directory is specified
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir(parents=True)
        nlp.to_disk(output_dir)
        print(f"Model saved to {output_dir}")
    
    return nlp

In [101]:
def filter_overlapping_entities(training_data):
    """
    Filter out overlapping entities from training data
    """
    cleaned_data = []
    
    for text, annotations in training_data:
        entities = annotations.get("entities", [])
        
        # Sort entities by start position
        entities.sort(key=lambda x: x[0])
        
        filtered_entities = []
        prev_end = -1
        
        for ent in entities:
            start, end, label = ent
            if start >= prev_end:
                filtered_entities.append(ent)
                prev_end = end
        
        cleaned_data.append((text, {"entities": filtered_entities}))
    
    return cleaned_data

In [38]:
convert_data = filter_overlapping_entities(convert_data)

In [102]:
trained_model = train_ner_model(
        convert_data,
        model_name="resume_ner",
        output_dir="./resume_ner_model",
        n_iter=50
    )

Created blank 'en' model


c:\Users\arshi\Downloads\Desktop\Bro-Project\SkillForge.ai\env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "   
Contact
www.linkedin.com/in/binod-kumar-
bhaga..." with entities "[(12, 61, 'LINKEDIN LINK'), (85, 102, 'SKILLS'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\arshi\Downloads\Desktop\Bro-Project\SkillForge.ai\env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "   
Contact
www.linkedin.com/in/madhu-
sudan-kamat..." with entities "[(12, 60, 'LINKEDIN LINK'), (83, 93, 'SKILLS'), (9...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\arshi\Downloads\Desktop\Bro-P

Beginning training...
Iteration 1/50, Loss: 63712.3281
Iteration 2/50, Loss: 34863.2031
Iteration 3/50, Loss: 32815.1875
Iteration 4/50, Loss: 31253.9141
Iteration 5/50, Loss: 28173.3086
Iteration 6/50, Loss: 26710.2070
Iteration 7/50, Loss: 23916.6172
Iteration 8/50, Loss: 23616.7305
Iteration 9/50, Loss: 22318.2344
Iteration 10/50, Loss: 21004.9395
Iteration 11/50, Loss: 19985.3730
Iteration 12/50, Loss: 19670.8633
Iteration 13/50, Loss: 18505.9922
Iteration 14/50, Loss: 18339.7520
Iteration 15/50, Loss: 18185.3906
Iteration 16/50, Loss: 17860.4922
Iteration 17/50, Loss: 17038.5176
Iteration 18/50, Loss: 16696.0020
Iteration 19/50, Loss: 16181.2549
Iteration 20/50, Loss: 16393.0938
Iteration 21/50, Loss: 16297.7168
Iteration 22/50, Loss: 15711.7676
Iteration 23/50, Loss: 15501.7803
Iteration 24/50, Loss: 15021.9697
Iteration 25/50, Loss: 15352.3594
Iteration 26/50, Loss: 14748.6025
Iteration 27/50, Loss: 14524.5469
Iteration 28/50, Loss: 14472.4922
Iteration 29/50, Loss: 14201.7637
I

In [103]:
test_text = "'Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-download&ikw=download-top&co=IN https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-download&ikw=download-top&co=IN   SKILLS  APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years), Algorithms (3 years)  LINKS  https://www.linkedin.com/in/govardhana-k-61024944/  ADDITIONAL INFORMATION  Technical Proficiency:  Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle PL-SQL programming, Sales Force with APEX. Tools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer, PL/SQL Developer, WinSCP, Putty Web Technologies: JavaScript, XML, HTML, Webservice  Operating Systems: Linux, Windows Version control system SVN & Git-Hub Databases: Oracle Middleware: Web logic, OC4J Product FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x  https://www.linkedin.com/in/govardhana-k-61024944/"
doc = trained_model(test_text)
print("\nTest prediction:")
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")
   
# skills = [ent.text for ent in doc.ents if ent.label_ == 'Skills']


Test prediction:
Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer (LOCATION)
Senior Consultant (WORKED AS)
Associate Consultant (WORKED AS)
3 years (YEARS OF EXPERIENCE)
5 years (YEARS OF EXPERIENCE)
5 years (YEARS OF EXPERIENCE)
3 years (YEARS OF EXPERIENCE)
HTML (SKILLS)


In [43]:
train_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [46]:
skills

['APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years), Algorithms (3 years)',
 'Technical Proficiency:  Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle PL-SQL programming, Sales Force with APEX. Tools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer, PL/SQL Developer, WinSCP, Putty Web Technologies: JavaScript, XML, HTML, Webservice  Operating Systems: Linux, Windows Version control system SVN & Git-Hub Databases: Oracle Middleware: Web logic, OC4J Product FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x  https://www.linkedin.com/in/govardhana-k-61024944/']

## Second way to create model using span

In [53]:
def convert_to_spans_format(training_data, nlp):
    """
    Convert entities to spans format with validation for:
    - Invalid offsets (start > end)
    - Offsets exceeding text length
    - Proper token alignment
    """
    spans_data = []
    error_count = 0
    
    for text, annotations in training_data:
        doc = nlp.make_doc(text)
        spans = []
        
        for ent in annotations.get("entities", []):
            if len(ent) != 3:
                continue
                
            start, end, label = ent
            
            # Validate offsets
            if start >= end:
                error_count += 1
                continue
                
            if end > len(text):
                error_count += 1
                continue
                
            span = doc.char_span(start, end, label=label)
            if span is not None:
                spans.append(span)
            else:
                error_count += 1
        
        spans_data.append((text, {"spans": spans}))
    
    if error_count > 0:
        print(f"Warning: Skipped {error_count} invalid entity spans")
    
    return spans_data

In [56]:
nlp1 = spacy.blank("en")

In [54]:
convert_data_span = filter_overlapping_entities(train_data)

In [57]:
train_data_span = convert_to_spans_format(convert_data,nlp1)

In [59]:
def train_ner_model_span(training_data, model_name="resume_ner", output_dir=None, n_iter=100):
    """Train with error handling for overlapping entities"""
    # Load or create model
    try:
        nlp = spacy.load(model_name)
        print(f"Loaded existing model '{model_name}'")
    except:
        nlp = spacy.blank("en")
        print(f"Created blank 'en' model")
    
    # Add NER pipeline
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    else:
        ner = nlp.get_pipe("ner")
    
    # Add labels
    for _, annotations in training_data:
        for ent in annotations.get("entities", []):
            ner.add_label(ent[2])
    
    # Training with error handling
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.initialize()
        
        print("Beginning training...")
        for itn in range(n_iter):
            random.shuffle(training_data)
            losses = {}
            
            batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                examples = []
                for text, annotations in batch:
                    doc = nlp.make_doc(text)
                    try:
                        example = Example.from_dict(doc, annotations)
                        examples.append(example)
                    except ValueError as e:
                        print(f"Skipping problematic example: {str(e)}")
                        continue
                
                if examples:  # Only update if we have valid examples
                    nlp.update(examples, drop=0.5, losses=losses, sgd=optimizer)
            
            print(f"Iteration {itn + 1}/{n_iter}, Loss: {losses.get('ner', 0):.4f}")
    
    if output_dir:
        output_dir = Path(output_dir)
        output_dir.mkdir(exist_ok=True)
        nlp.to_disk(output_dir)
        print(f"Model saved to {output_dir}")
    
    return nlp

In [60]:
trained_model2 = train_ner_model_span(
    train_data_span,
    model_name="resume_ner2",
    output_dir="./resume_ner_model2",
    n_iter=50
)

Created blank 'en' model
Beginning training...
Skipping problematic example: [E879] Unexpected type for 'spans' data. Provide a dictionary mapping keys to a list of spans, with each span represented by a tuple (start_char, end_char). The tuple can be optionally extended with a label and a KB ID.
Skipping problematic example: [E879] Unexpected type for 'spans' data. Provide a dictionary mapping keys to a list of spans, with each span represented by a tuple (start_char, end_char). The tuple can be optionally extended with a label and a KB ID.
Skipping problematic example: [E879] Unexpected type for 'spans' data. Provide a dictionary mapping keys to a list of spans, with each span represented by a tuple (start_char, end_char). The tuple can be optionally extended with a label and a KB ID.
Skipping problematic example: [E879] Unexpected type for 'spans' data. Provide a dictionary mapping keys to a list of spans, with each span represented by a tuple (start_char, end_char). The tuple can be

In [87]:
test_text = "Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-download&ikw=download-top&co=IN https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-download&ikw=download-top&co=IN   SKILLS  APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years), Algorithms (3 years)  LINKS  https://www.linkedin.com/in/govardhana-k-61024944/  ADDITIONAL INFORMATION  Technical Proficiency:  Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle PL-SQL programming, Sales Force with APEX. Tools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer, PL/SQL Developer, WinSCP, Putty Web Technologies: JavaScript, XML, HTML, Webservice  Operating Systems: Linux, Windows Version control system SVN & Git-Hub Databases: Oracle Middleware: Web logic, OC4J Product FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x  https://www.linkedin.com/in/govardhana-k-61024944/"
doc = trained_model(test_text)
print("\nTest prediction:")
# for ent in doc.ents:
#     print(f"{ent.text} ({ent.label_})")
   
skills2 = [ent.text for ent in doc.ents if ent.label_ == 'Skills']


Test prediction:


In [88]:
skills2

['APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years), Algorithms (3 years)',
 'Technical Proficiency:  Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle PL-SQL programming, Sales Force with APEX. Tools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer, PL/SQL Developer, WinSCP, Putty Web Technologies: JavaScript, XML, HTML, Webservice  Operating Systems: Linux, Windows Version control system SVN & Git-Hub Databases: Oracle Middleware: Web logic, OC4J Product FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x  https://www.linkedin.com/in/govardhana-k-61024944/']

In [63]:
print('model creation failed')

model creation failed


In [64]:
os.getcwd()

'c:\\Users\\arshi\\Downloads\\Desktop\\Bro-Project\\SkillForge.ai'

In [67]:
from pathlib import Path
from PyPDF2 import PdfReader 

In [81]:
with open("data\\Arshid.pdf",'rb') as file:
    pdf_reader = PdfReader(file)
    full_text = ""
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:  # Only add if text exists
            full_text += page_text + "\n"  # Add newline between pages
    
    # Print the extracted text
    print(full_text)

Arshid T M
DATA SCIENTIST
♂¶ap-¶arker-altKannur,Kerala /envel⌢pearshidtm2001@gmail.com ♂phone-alt+91 8078196576 < / > Leetcode /linkedin-inArshid T M
/githubArshidtm
Profile Summary
Data Science and Machine Learning enthusiast with expertise in building predictive models, deep learning,
and NLP. Skilled in data preprocessing, feature engineering, and EDA. Proficient in Python, TensorFlow, and
Scikit-Learn for model development and deployment. Passionate about leveraging AI to solve real-world problems
and drive business growth.
Skills
◦Programming Language : Python
◦Data Analysis & Visualization : Pandas, NumPy, Matplotlib, Seaborn, Tableau
◦Database Management : PostgreSQL
◦Machine Learning :
– Regression : Linear Regression, Random Forest Regressor
– Classification : KNN, Naive Bayes, SVM, Decision Trees, Random Forest Classifier
– Ensemble Learning : Bagging, XgBoost
– Clustering : K-Means, DBSCAN
– Advanced Topics : Natural Language Processing (NLP), Deep Learning
Projects
Sentimen

In [104]:
doc = trained_model(full_text)
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")

◦Programming Language : Python (SKILLS)
◦Data Analysis & Visualization : Pandas (CERTIFICATION)
NumPy (SKILLS)
Matplotlib (SKILLS)
◦Machine Learning :
– Regression : Linear Regression, Random Forest Regressor
– Classification : KNN, Naive Bayes, SVM, Decision Trees, Random Forest Classifier
– Ensemble Learning : Bagging, XgBoost
– Clustering : K-Means, DBSCAN
– Advanced Topics : Natural Language Processing (NLP), Deep Learning
Projects (CERTIFICATION)
Sentiment Analysis GitHub
Developed a Machine learning -based sentiment analysis project that applies NLP techniques like tokenization,
stemming, and vectorization (TF-IDF & Word2Vec). Trained using Random Forest and Multinomial Na ¨ıve
Bayes , achieving 96% accuracy with Grid Search tuning. Developed as a web application using Flask for real-
time predictions.
 (CERTIFICATION)
◦Machine Learning Models : Implemented Random Forest & Multinomial Na¨ ıve Bayes for text classifica-
tion. (CERTIFICATION)
Movie Recommendation System – Content-B

In [155]:
skills = [ent.text for ent in doc.ents if ent.label_ == 'SKILLS' or ent.label_ == "CERTIFICATION" or ent.label_ == "DEGREE"]

In [156]:
skills = " ".join(skills)

In [157]:
skills

'◦Programming Language : Python ◦Data Analysis & Visualization : Pandas NumPy Matplotlib ◦Machine Learning :\n– Regression : Linear Regression, Random Forest Regressor\n– Classification : KNN, Naive Bayes, SVM, Decision Trees, Random Forest Classifier\n– Ensemble Learning : Bagging, XgBoost\n– Clustering : K-Means, DBSCAN\n– Advanced Topics : Natural Language Processing (NLP), Deep Learning\nProjects Sentiment Analysis GitHub\nDeveloped a Machine learning -based sentiment analysis project that applies NLP techniques like tokenization,\nstemming, and vectorization (TF-IDF & Word2Vec). Trained using Random Forest and Multinomial Na ¨ıve\nBayes , achieving 96% accuracy with Grid Search tuning. Developed as a web application using Flask for real-\ntime predictions.\n ◦Machine Learning Models : Implemented Random Forest & Multinomial Na¨ ıve Bayes for text classifica-\ntion. Movie Recommendation System – Content-Based Filtering GitHub Mini Project Master of Science in Data Science and Busin

In [109]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [110]:
embeddings = HuggingFaceEmbeddings()

c:\Users\arshi\Downloads\Desktop\Bro-Project\SkillForge.ai\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data\\Arshid.pdf

In [116]:
vector_store = FAISS.load_local(
    folder_path="research\\job_vector_db",  # Path to saved folder
    embeddings=embeddings,
    allow_dangerous_deserialization = True
)

In [158]:
system_prompt = ("""
You are a friendly and knowledgeable AI Career Mentor. Your role is to analyze skill gaps between a user's extracted resume skills and job descriptions from LinkedIn. You will provide insightful and practical recommendations to help users improve their qualifications and bridge any skill gaps. Be honest, supportive, and solution-oriented.

Instructions:
1. If the context provided is a casual greeting such as "Hi", "Hello", or "How are you?", respond briefly by introducing who you are. Avoid adding any unnecessary or unrelated information
2. Carefully analyze the context provided, which includes the user's extracted skills and job requirements from the vector store.
3 Identify missing or underdeveloped skills and suggest actionable steps to bridge these gaps. 
   - Provide resources like courses, certifications, or projects.
   - Recommend practical experience or networking opportunities where relevant.
4 Offer clear, concise, and easy-to-understand responses tailored to the user's career growth.
5 If there is insufficient information to answer accurately, admit it and suggest rephrasing the query or providing additional details.
6. Provide alternative options where possible and avoid making up information or giving speculative responses.
7. Do not treat the vector store data as a specific job description — it is intended for understanding market-relevant skills.
Context: {context}
""")


In [159]:
retriever=vector_store.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [160]:
contextualize_q_system_prompt  = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "Formulate a standalone query which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return as it is."
)

In [161]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain,create_history_aware_retriever

In [162]:
from langchain_groq import ChatGroq


llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    
    
)

In [163]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ('human',"{input}"),
    ]
)

In [164]:
history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)


In [165]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [166]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [167]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [171]:
response = conversational_rag_chain.invoke(
    {"input": "what are the relevnt skills required to the data science job role "},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [172]:
print(response)

Based on the provided context, the relevant skills required for a data science job role are:

1. **Technical Skills**:
	* Proficiency in Microsoft Office applications, particularly Excel (e.g., pivot tables, advanced functions, formulas, filtering)
	* Database skills (e.g., SQL)
	* Working knowledge of data management and analytics platforms (e.g., SQL Server, SSIS, Power BI, Google Analytics)
	* Familiarity with ERP systems and warehouse management tools
2. **Data Analysis and Visualization**:
	* Ability to collect and synthesize information, making it relevant, understandable, and actionable for key stakeholders
	* Strong understanding of Analytics and Visualization techniques
	* Demonstrated ability in statistical analysis, predictive modeling, and machine learning techniques
3. **Communication and Collaboration**:
	* Excellent Communication & Coordination skills, with a strong command over verbal and written English
	* Ability to communicate trends and their implications clearly an

In [147]:
old1 = response

In [149]:
print(response)

Based on the provided context, I'll analyze the skill gaps between your extracted resume skills and the job requirements for an AI/ML Developer position.

**Strengths:**

1. Python programming: You have a solid foundation in Python, which is essential for the role.
2. Machine Learning: You have experience with various machine learning algorithms, including regression, classification, ensemble learning, and clustering.
3. Data Analysis & Visualization: You're familiar with popular libraries like Pandas, NumPy, and Matplotlib.
4. NLP: You've worked on a sentiment analysis project, demonstrating your understanding of NLP techniques like tokenization, stemming, and vectorization.

**Skill Gaps:**

1. **Large Language Models (LLM)**: The job requires experience with LLMs, which is not explicitly mentioned in your skills. You may want to explore courses or projects related to LLMs, such as transformer-based models.
2. **Retrieval-Augmented Generation (RAG) frameworks**: This is another area 

In [173]:
user_input = "how can i boost up my skills"

In [174]:
response2 = conversational_rag_chain.invoke(
    {"input": user_input},
    config={
        "configurable": {"session_id": "abc123"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)["answer"]

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jp4qhsk5enfvzcds5rj2hy78` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98562, Requested 1752. Please try again in 4m31.05s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [154]:
print(response2)

To enhance your understanding of Large Language Models (LLMs), I recommend working on projects that involve building, fine-tuning, and applying LLMs to real-world problems. Here are some project ideas to get you started:

1. **Text Classification with LLMs**:
	* Use a pre-trained LLM like BERT, RoBERTa, or DistilBERT to classify text into categories like spam vs. non-spam emails, positive vs. negative movie reviews, or product reviews.
	* Fine-tune the LLM on your dataset and evaluate its performance.
2. **Language Translation with LLMs**:
	* Use a pre-trained LLM like transformer-based models (e.g., Marian, Moses) to translate text from one language to another.
	* Experiment with different architectures, such as sequence-to-sequence or attention-based models.
3. **Sentiment Analysis with LLMs**:
	* Use a pre-trained LLM to analyze the sentiment of text data, such as movie reviews, product reviews, or social media posts.
	* Compare the performance of different LLMs, such as BERT, RoBER